<a href="https://colab.research.google.com/github/albordunos/topic_modeling/blob/main/LDA_TG_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка необходимых библиотек
!pip install pandas xlrd nltk gensim pyLDAvis chardet matplotlib wordcloud razdel pymorphy2 stopwords

# Импорт необходимых библиотек
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Импортируем лемматизатор
import re
import string
from gensim import corpora
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import chardet
from gensim.models import Phrases  # Импортируем класс Phrases
from gensim.models.coherencemodel import CoherenceModel  # Импортируем CoherenceModel
import matplotlib.pyplot as plt  # Для визуализации
from wordcloud import WordCloud
from razdel import tokenize
from nltk.corpus import stopwords
import pymorphy2
import stopwords  # Импортируем библиотеку stopwords

# Загрузка ресурсов NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
# === 1. Загрузка данных ===
# Укажите путь к файлу .csv
file_path = '/content/MID1.csv'

# Определение кодировки файла
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read(10000))
    detected_encoding = result['encoding']
    print(f"Определённая кодировка: {detected_encoding}")

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Чтение CSV-файла с использованием определенной кодировки и обработкой некорректных строк
df = pd.read_csv(file_path, encoding=detected_encoding, sep=';', on_bad_lines='skip')

# Вывод первых строк для проверки
print(df.head())

In [ ]:
# Укажите название столбца с текстом
text_column = 'message'  # Замените на реальное название столбца

# Вывод информации о количестве строк и слов в исходных данных
num_rows = df.shape[0]
num_words = df[text_column].str.split().str.len().sum()  # Считаем общее количество слов в указанном столбце

print(f'Количество строк в исходных данных: {num_rows}')
print(f'Количество слов в исходных данных: {num_words}')

 # === 2. Предобработка текста ===
# Определение своих стоп-слов
custom_stop_words = ['мид', 'россия', 'россии', 'который', 'которые', 'которых', 'также', 'именно', 'включая', 'сегодня', 'лавров', 'захарова', 'минимтр', 'иностранных', 'дел', 'год', 'российский', 'федерация']

# Получаем стоп-слова для русского языка из библиотеки stopwords
russian_stop_words = stopwords.get_stopwords('ru')

# Объединяем с пользовательскими стоп-словами
stop_words = set(russian_stop_words + custom_stop_words)

# Инициализация лемматизатора
morph = pymorphy2.MorphAnalyzer()  # Инициализация лемматизатора pymorphy2

def preprocess_text(text):
    if pd.isna(text):  # Проверка на NaN
        return []

    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Удаляем URL

    # Удаление пунктуации и чисел, а также лишних пробелов
    text = re.sub(r'[^\w\s]', ' ', text)  # Заменяем пунктуацию на пробел
    text = re.sub(r'\d+', '', text)  # Удаляем числа
    text = re.sub(r'\s+', ' ', text)  # Заменяем несколько пробелов на один
    text = text.strip()  # Удаление пробелов в начале и конце строки

    # Токенизация текста
    tokens = word_tokenize(text)

    # Лемматизация токенов и фильтрация: удаление неалфавитных и стоп-слов
    filtered_tokens = [
        morph.parse(token.text)[0].normal_form
        for token in tokenize(text)
        if token.text.isalnum() and token.text not in stop_words and len(token.text) > 2
    ]

    return filtered_tokens  # Возвращаем список токенов


# Применение функции очистки текста к указанному столбцу
df['cleaned_text'] = df[text_column].astype(str).apply(preprocess_text)

# === 3. Создание n-грамм ===
# Создаем модель биграмм
bigram_model = Phrases(df['cleaned_text'], min_count=5, threshold=10)

# Применяем биграммную модель к очищенным токенам
df['bigrams'] = df['cleaned_text'].apply(lambda tokens: bigram_model[tokens])

# === 4. Создание корпуса и словаря ===
# Преобразуем очищенные тексты в список токенов
tokenized_texts = df['cleaned_text'].tolist()  # Получаем список списков токенов
tokenized_bigrams = df['bigrams'].tolist()  # Получаем список биграмм

# Создаем словарь (gensim Dictionary)
dictionary = corpora.Dictionary(tokenized_texts + tokenized_bigrams)

# Удаление редких и слишком частых слов (по желанию)
dictionary.filter_extremes(no_below=5, no_above=0.7)

# Создаем корпус (список мешков слов)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts + tokenized_bigrams]

# === 5. TF-IDF модель ===
# Применяем TF-IDF модель к корпусу
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# Вывод статистики и первых строк для проверки
num_docs = len(tokenized_texts)
num_words = sum([len(tokens) for tokens in tokenized_texts])
print(f"Количество документов (строк): {num_docs}")
print(f"Общее количество слов: {num_words}")
print(df[['message', 'cleaned_text', 'bigrams']].head())

In [ ]:
#====6 обучение модели====
# Параметры для подбора количества тем
num_topics = int(input("Введите количество тем: "))

# Создание модели LDA с заданным количеством тем
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     passes=5)

# Функция для получения доминирующей темы для документа
def get_dominant_topic(lda_model, doc_topics):
    return max(doc_topics, key=lambda item: item[1])

# Получение тем для всех документов
doc_topics = lda_model.get_document_topics(corpus)

# Вывод результатов
for idx, topic in lda_model.print_topics(-1):
    print('Тема: {}'.format(idx))
    print('Слова: {}'.format(topic))

    # Получение документов, относящихся к теме
    doc_topics = lda_model.get_document_topics(corpus)
    docs = [doc_id for doc_id, topic_values in enumerate(doc_topics) if max(topic_values, key=lambda x: x[1])[0] == idx]

    # Вывод номеров строк для темы
    print('Документы (строки):', docs)
    print()



In [ ]:
# === 7.1 Визуализация тем с помощью pyLDAvis ===
# Подготовка данных для визуализации
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_vis)

# --- 7.2. Распределение тем (pyLDAvis) ---
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

In [ ]:
# === 8. Получение наиболее вероятных слов для каждой темы ===
topics_words = lda_model.show_topics(formatted=False)

# === 9. Облака слов для каждой темы ===
# Создание облака слов для каждой темы
for topic_id, topic in topics_words:
    word_freq = {word: freq for word, freq in topic}

    # Генерация облака слов
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

    # Визуализация облака слов
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Облако слов для темы {topic_id}')
    plt.show()

In [ ]:
#=== 10. Сохранение результатов ===
# Сохранение модели и словаря (по желанию)
lda_model.save('/content/lda_model.gensim')
dictionary.save('/content/dictionary.gensim')

# Сохранение очищенного текста
df.to_csv('/content/cleaned_texts.csv', index=False)
print("Результаты сохранены в файл: /content/cleaned_texts.csv")

In [ ]:
!pip install pandas openpyxl  # Для сохранения в .xlsx


# Функция для получения доминирующей темы для документа
def get_dominant_topic(doc_topics):
    return max(doc_topics, key=lambda item: item[1])

# Получение тем для всех документов
doc_topics = lda_model.get_document_topics(corpus)

# Список для хранения данных
data = []

# Вывод результатов и сбор данных
for idx, topic in lda_model.print_topics(-1):
    print('Тема: {}'.format(idx))
    print('Слова: {}'.format(topic))

    # Получение документов, относящихся к теме
    docs = [doc_id for doc_id, topic_values in enumerate(doc_topics) if get_dominant_topic(topic_values)[0] == idx]

    # Добавляем информацию о теме и связанных документах в список
    for doc_id in docs:
        data.append({'Topic ID': idx, 'Words': topic, 'Document ID': doc_id})

    # Вывод номеров строк для темы
    print('Документы (строки):', docs)
    print()

# Создание DataFrame из собранных данных
df = pd.DataFrame(data)

# Сохранение DataFrame в Excel файл
df.to_excel('/content/MID1.xlsx', index=False)  # Для .xlsx

print("Данные успешно сохранены в /content/MID1.xlsx")
